<a href="https://colab.research.google.com/github/jhzhang07/tesP/blob/master/task_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install torchinfo

In [3]:
# @title Default title text
import os.path

import torch
import sys
sys.path.append('/content/drive/MyDrive/CAS771-Project-main')
import matplotlib.pyplot as plt
from arguments import Arguments
from dataset import DataLoader, create_combined_loader
from evaluate import Evaluator
from TS_model import ClassifierModel, ConcatModel, EnsembleModel, ClassifierModel_squeezenet1_0, ClassifierModel_2
from train import Trainer
import torchvision
from torchinfo import summary



DATA_MODEL_PATH = '/content/drive/MyDrive/CAS771-Project-main/models'

MODEL_PARAMETER_LIMIT = 88 * 1e6


def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

classifier1_args = Arguments(
    train_data='TaskB_fusion_train.pth',
    test_data='TaskB_fusion_test.pth',
    # train_data='train_dataB_model_1.pth',
    # test_data='val_dataB_model_1.pth',
    subset=[173, 137, 34, 159, 201],
    n_epochs=20,
    batch_size=64,
    lr=1e-2,
)

classifier2_args = Arguments(
    train_data='TaskB_fusion_train.pth',
    test_data='TaskB_fusion_test.pth',
    # train_data='train_dataB_model_2.pth',
    # test_data='val_dataB_model_2.pth',
    subset=[34, 202, 80, 135, 24],
    n_epochs=20,
    batch_size=64,
    lr=1e-2,
)

classifier3_args = Arguments(
    train_data='TaskB_fusion_train.pth',
    test_data='TaskB_fusion_test.pth',
    # train_data='train_dataB_model_3.pth',
    # test_data='val_dataB_model_3.pth',
    subset=[173, 202, 130, 124, 125],
    n_epochs=20,
    batch_size=64,
    lr=1e-2,
)


def main():
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    models = []
    test_datasets = []
    train_datasets = []


    # Train and evaluate three classifiers
    for i, args in enumerate([classifier1_args,
                              classifier2_args,
                              classifier3_args, ]):
        model_id = i + 1

        dataloader = DataLoader(args=args,
                                root=DATA_MODEL_PATH)
        train_dataset, train_loader = dataloader.train_dataset_loader()
        test_dataset, test_loader = dataloader.test_dataset_loader()


        model = ClassifierModel_2(num_classes=len(args.subset)).to(device)
        # model = ClassifierModel_squeezenet1_0(num_classes=len(args.subset)).to(device)
        trainer = Trainer(args=args,
                          loader=train_loader,
                          device=device)

        print(f">>> Start training model for classifier{model_id}")

        print(summary(model, input_size=(1, 3, 224, 224)))



        trainer.train(model,
                      save_path=f'./model{model_id}_trained.pth',
                      # comment out the following line to always train before evaluating.
                      # always_load=True,
                      )
        assert count_parameters(model) < MODEL_PARAMETER_LIMIT

        evaluator = Evaluator(loader=test_loader,
                              device=device)
        test_acc = evaluator.evaluate(model)
        print(f"Test accuracy for classifier{model_id} on subset{model_id}: {test_acc:.2f}%")

        # add to globals
        models.append(model)
        test_datasets.append(test_dataset)
        train_datasets.append(train_dataset)

    # Unified Model and Evaluation
    ensemble_model = EnsembleModel(models[0], models[1], models[2])

    # for param in ensemble_model.parameters():
    # param.requires_grad = False

    # for param in ensemble_model.classifier.parameters():
    # param.requires_grad = True

    ensemble_model = ensemble_model.to(device)

    classifier_unified = Arguments(
    train_data='TaskB_fusion_train.pth',
    test_data='TaskB_fusion_test.pth',
    # subset=[173, 137, 34, 159, 201, 80, 135],
    subset=[173, 137, 34, 159, 201,80, 135, 24, 202, 130, 124, 125],
    n_epochs=20,
    batch_size=64,
    lr=1e-2,
)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")


    # Train and evaluate uni-classifiers
    for i, args in enumerate([classifier_unified
                           ]):
        model_id = i + 1

        dataloader = DataLoader(args=args,
                                root=DATA_MODEL_PATH)
        train_dataset, train_loader = dataloader.train_dataset_loader()
        test_dataset, test_loader = dataloader.test_dataset_loader()

        # model = ClassifierModel(num_classes=len(args.subset)).to(device)

        trainer = Trainer(args=args,
                          loader=train_loader,
                          device=device)

        print(f">>> Start training model for unified_classifier")
        print(summary(ensemble_model, input_size=(1, 3, 224, 224)))
        trainer.train(ensemble_model,
                      save_path=f'./unified_model_trained.pth',
                      # comment out the following line to always train before evaluating.
                      # always_load=True,
                      )
        assert count_parameters(ensemble_model) < MODEL_PARAMETER_LIMIT

        print(count_parameters(ensemble_model))
        evaluator = Evaluator(loader=test_loader,
                              device=device)
        test_acc = evaluator.evaluate(ensemble_model)
        print(f"Test accuracy for unified_classifier on subset: {test_acc:.2f}%")




if __name__ == "__main__":
    main()

Using device: cpu
Subset train samples: 2500


Downloading: "https://download.pytorch.org/models/mobilenet_v3_large-5c1a4163.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v3_large-5c1a4163.pth


Subset test samples: 200


100%|██████████| 21.1M/21.1M [00:00<00:00, 70.4MB/s]


>>> Start training model for classifier1
Layer (type:depth-idx)                                  Output Shape              Param #
ClassifierModel_2                                       [1, 5]                    --
├─MobileNetV3: 1-1                                      [1, 5]                    --
│    └─Sequential: 2-1                                  [1, 960, 7, 7]            --
│    │    └─Conv2dNormActivation: 3-1                   [1, 16, 112, 112]         (464)
│    │    └─InvertedResidual: 3-2                       [1, 16, 112, 112]         (464)
│    │    └─InvertedResidual: 3-3                       [1, 24, 56, 56]           (3,440)
│    │    └─InvertedResidual: 3-4                       [1, 24, 56, 56]           (4,440)
│    │    └─InvertedResidual: 3-5                       [1, 40, 28, 28]           (10,328)
│    │    └─InvertedResidual: 3-6                       [1, 40, 28, 28]           (20,992)
│    │    └─InvertedResidual: 3-7                       [1, 40, 28, 28]     